In [1]:
import pandas as pd
import numpy as np

# Random seed for reproducibility
seed = 202
np.random.seed(seed)

# Import the train set and test set
train_data = pd.read_csv("trainset.csv", delimiter=",")
test_data = pd.read_csv("testset.csv", delimiter=",")
train_data2 = pd.read_csv("trainset.csv", delimiter=",")
test_data2 = pd.read_csv("testset.csv", delimiter=",")

In [2]:
def normalize_costumer_age(dataset):
    #Take all unique Cosumer_Age, assigns to each age alphabetically ordered a mean age
    customer_age_sort = sorted(dataset['CustomerAge'].dropna().unique())

    mapping = {}
    for idx, val in enumerate(customer_age_sort):
        mapping[val] = int(15+idx*10)
        
    ages = dataset['CustomerAge']
    ages.replace(mapping, inplace=True)

    #print(mapping, regions)
    ages = np.asarray(ages)
    ages = pd.DataFrame({'CustomerAge': ages})
    
     # Remove the previous CustomerAge and then insert the new CustomerAge
    dataset = dataset.drop('CustomerAge', axis=1)
    dataset = dataset.join(ages)
    return dataset

In [4]:
def normalize_region(dataset):
    #Take all unique Regions, assigns to each region alphabetically ordered a monotonic growing number
    regions_sort = sorted(dataset['Region'].dropna().unique())
    #regions_len = len(regions_sort) #20 Regions

    mapping = {}
    for idx, val in enumerate(regions_sort):
        mapping[val] = int(idx)

    regions = dataset['Region']
    regions.replace(mapping, inplace=True)

    #print(mapping, regions)
    regions = np.asarray(regions)
    regions = pd.DataFrame({'Region': regions})
    
    # Remove the previous Region columns and then insert the new Region
    dataset = dataset.drop('Region', axis=1)
    dataset = dataset.join(regions)
    return dataset

In [6]:
def normalize_province(dataset):
    #Take all unique Provinces, assigns to each province alphabetically ordered a monotonic growing number
    provinces_sort = sorted(dataset['Province'].dropna().unique())
    #provinces_len = len(provinces_sort) #110 Provinces

    mapping = {}
    for idx, val in enumerate(provinces_sort):
        mapping[val] = idx

    provinces = dataset['Province']
    provinces.replace(mapping, inplace=True)


    #print(provinces_sort, provinces_len)
    #print(mapping, provinces)
    provinces = np.asarray(provinces)
    provinces = pd.DataFrame({'Province': provinces})
    
    # Remove the previous Province columns and then insert the new Province
    dataset = dataset.drop('Province', axis=1)
    dataset = dataset.join(provinces)
    return dataset

In [5]:
def normalize_product(dataset):
    if('Product' in dataset.columns):
        #Take all unique Products, assigns to each product alphabetically ordered a monotonic growing number
        products_sort = sorted(dataset['Product'].dropna().unique())

        mapping = {}
        for idx, val in enumerate(products_sort):
            mapping[val] = int(idx)

        products = dataset['Product']
        products.replace(mapping, inplace=True)

        #print(mapping, products)
        products = np.asarray(products)
        products = pd.DataFrame({'Product': products})

        # Remove the previous Product columns and then insert the newProduct
        dataset = dataset.drop('Product', axis=1)
        dataset = dataset.join(products)
    return dataset

In [7]:
def drop_useless_columns(dataset):
    if('DataAllowanceOneShot' in dataset.columns and 'EstimatedDevicePrice' in dataset.columns):
        dataset = dataset.drop('DataAllowanceOneShot', axis=1)
        dataset = dataset.drop('EstimatedDevicePrice', axis=1)
    return dataset

In [8]:
def normalize_data_set(dataset):
    dataset = normalize_costumer_age(dataset)
    dataset = normalize_region(dataset)
    dataset = normalize_province(dataset)
    dataset = normalize_product(dataset)
    dataset = drop_useless_columns(dataset)
    return dataset

In [9]:
# Fill the missing values with the mean for each column
def fillna_mean_value(dataset):
    # fill missing values with mean column values
    dataset.fillna(dataset.mean(), inplace=True)
    return dataset

In [10]:
train_data = normalize_data_set(train_data)
test_data = normalize_data_set(test_data)

In [294]:
# train_data.to_csv('normalized_train_dataset.csv', index = False, encoding='utf-8')
# test_data.to_csv('normalized_test_dataset.csv', index = False, encoding='utf-8')

In [11]:
train_data.columns

Index(['ID', 'DeviceFlag4G', 'DataArpu', 'DataAllowanceContinuous',
       'DeviceFlagSmartphone', 'MonthlyVoiceTrafficCount',
       'MonthlySMSTrafficCount', 'MonthlyDataTraffic', 'CustomerGender',
       'CustomerExpatriate', 'ZipCode', 'ChurnScore',
       'AirportConnectionsDuration', 'AirportConnectionsCount',
       'StationConnectionsDuration', 'StationConnectionsCount',
       'ParkingConnectionsDuration', 'ParkingConnectionsCount',
       'File-Transfer', 'Games', 'Instant-Messaging-Applications', 'Mail',
       'Music-Streaming', 'Network-Operation', 'P2P-Applications', 'Security',
       'Streaming-Applications', 'Terminals', 'Unclassified', 'VoIP',
       'Web-Applications', 'CustomerAge', 'Region', 'Province', 'Product'],
      dtype='object')

In [12]:
regions_sort = sorted(train_data['Region'].dropna().unique())
mapping = {}
for idx, val in enumerate(regions_sort):
    mapping[val] = int(idx)
mapping ={v: k for k, v in mapping.items()}

In [18]:
def fill_nan(field, data):
    nnan = data.isnull()[field].sum()
    field_sort = sorted(data[field].dropna().unique())
    table_temp = pd.DataFrame()

    field_id = data[['ID',field]]

    field_count = field_id.groupby(field).count()
    field_count = pd.DataFrame({'Count': field_count['ID']})
    table_temp = field_count

    field_per = round(field_count/(len(data[field].dropna(axis=0)))*100)
    field_per = pd.DataFrame({'Percentage': field_per['Count']})
    table_temp = table_temp.join(field_per)

    field_2badd = round(table_temp['Percentage']/100*nnan)
    field_2badd = pd.DataFrame({'FieldToAdd': field_2badd})
    table_temp = table_temp.join(field_2badd)
    
    import math
    test = data
    le = len(test)
    ct = table_temp['FieldToAdd'].values[0]
    j = 0
    for i in range(le):
        if(math.isnan(test[field][i])):
            test[field][i] = table_temp.index[j]
            #print("ct: " + str(ct) + " region: " + str(count.index[j]) + " J: " + str(j))
            ct -=1
            if(ct <= 0.0 and j <= len(table_temp['FieldToAdd'])):
                j += 1
                ct = table_temp['FieldToAdd'].values[j]
    return test

In [14]:
def fill_nan_province(data):
    final_train = data.sort_values(by='Region')
    final_province = final_train['Province'].fillna(method='ffill')
    final_province
    final_train = final_train.drop('Province', axis=1)
    final_train = final_train.join(final_province)
    return final_train

In [122]:
# new_train = fill_nan('Region', train_data)
# new_train = fill_nan_province(new_train)
#new_train = fill_nan('CustomerAge', new_train)
#new_train['CustomerAge']

In [224]:
new_train.isnull().sum().head(5)

ID                            0
DeviceFlag4G                  0
DataArpu                   2225
DataAllowanceContinuous    1180
DeviceFlagSmartphone          0
dtype: int64

In [148]:
len(train_data)

9567

In [149]:
test_age, train_age = test_data.CustomerAge.value_counts(),train_data.CustomerAge.value_counts()
round(test_age[test_age.index[:]]/len(test_data)*100),round(train_age[train_age.index[:]]/len(train_data)*100)

(45.0    24.0
 55.0    21.0
 35.0    19.0
 25.0    16.0
 65.0    10.0
 75.0     3.0
 15.0     2.0
 85.0     1.0
 Name: CustomerAge, dtype: float64, 45.0    24.0
 55.0    21.0
 35.0    19.0
 25.0    16.0
 65.0    10.0
 75.0     4.0
 15.0     2.0
 85.0     1.0
 Name: CustomerAge, dtype: float64)

In [12]:
train_data.head()

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,Security,Streaming-Applications,Terminals,Unclassified,VoIP,Web-Applications,CustomerAge,Region,Province,Product
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,45.0,16.0,14.0,0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,0.002620,0.071551,0.0,0.001896,0.000058,0.775410,25.0,NaN,NaN,0
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,0.000228,0.099438,0.0,0.012055,0.003579,0.499158,35.0,NaN,NaN,0
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,0.000052,0.041474,0.0,0.051692,0.013263,0.301528,55.0,8.0,15.0,2
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,0.000777,0.005975,0.0,0.000857,0.019149,0.926208,65.0,19.0,103.0,0


In [20]:
train_ooe = train_data
train_ooe.Region.head()

0    16.0
1     NaN
2     NaN
3     8.0
4    19.0
Name: Region, dtype: float64

In [21]:
# train_ooe = train_data
# region_ooe = train_ooe['Region']
# region_ooe.values

# nord_regions = [4,5,7,8,11,16,18,19]
# nord = []
# centro_regions = [0,6,9,10,15,17]
# centro = []
# sud_regions = [1,2,3,12,13,14]
# sud = []

# for i in region_ooe.values:
#     if i in nord_regions:
#         nord.append(1)
#         centro.append(0)
#         sud.append(0)
#     elif i in centro_regions:
#         nord.append(0)
#         centro.append(1)
#         sud.append(0)    
#     elif i in sud_regions:
#         nord.append(0)
#         centro.append(0)
#         sud.append(1)
#     else:
#         nord.append(np.nan)
#         centro.append(np.nan)
#         sud.append(np.nan)
        
# no = pd.DataFrame({'North_Region' : nord})
# ce = pd.DataFrame({'Middle_Region' : centro})
# su = pd.DataFrame({'South_Region' : sud})

# train_ooe = train_ooe.join(no)
# train_ooe = train_ooe.join(ce)
# train_ooe = train_ooe.join(su)

train_ooe = train_data

region_ooe = sorted(train_data['Region'].dropna().unique())

nord_regions = [4,5,7,8,11,16,18,19]
nord = []
centro_regions = [0,6,9,10,15,17]
centro = []
sud_regions = [1,2,3,12,13,14]
sud = []

mapping = {}
for idx, val in enumerate(region_ooe):
    #print(val, idx)
    if val in nord_regions:
        mapping[val] = 2
    elif val in centro_regions:
        mapping[val] = 3
    elif val in sud_regions:
        mapping[val] = 4
    else:
        mapping[np.nan] = np.nan
            
#region_ooe = region_ooe.replace(mapping, inplace=True)
mapping

{0.0: 3,
 1.0: 4,
 2.0: 4,
 3.0: 4,
 4.0: 2,
 5.0: 2,
 6.0: 3,
 7.0: 2,
 8.0: 2,
 9.0: 3,
 10.0: 3,
 11.0: 2,
 12.0: 4,
 13.0: 4,
 14.0: 4,
 15.0: 3,
 16.0: 2,
 17.0: 3,
 18.0: 2,
 19.0: 2}

In [22]:
old_reg = pd.DataFrame({'RegionsOOE': train_ooe.Region})
train_ooe = train_ooe.join(old_reg)
train_ooe.RegionsOOE = train_ooe.Region.replace(mapping)

train_ooe.head(5)

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,Streaming-Applications,Terminals,Unclassified,VoIP,Web-Applications,CustomerAge,Region,Province,Product,RegionsOOE
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,0.000000,0.0,0.000000,0.000000,0.000000,45.0,16.0,14.0,0,2.0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,0.071551,0.0,0.001896,0.000058,0.775410,25.0,NaN,NaN,0,NaN
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,0.099438,0.0,0.012055,0.003579,0.499158,35.0,NaN,NaN,0,NaN
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,0.041474,0.0,0.051692,0.013263,0.301528,55.0,8.0,15.0,2,2.0
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,0.005975,0.0,0.000857,0.019149,0.926208,65.0,19.0,103.0,0,2.0


In [23]:
train_ooe['RegionsOOE']

0       2.0
1       NaN
2       NaN
3       2.0
4       2.0
5       NaN
6       4.0
7       2.0
8       3.0
9       2.0
10      3.0
11      NaN
12      3.0
13      2.0
14      3.0
15      2.0
16      4.0
17      2.0
18      4.0
19      2.0
20      3.0
21      3.0
22      2.0
23      4.0
24      4.0
25      2.0
26      3.0
27      2.0
28      2.0
29      4.0
       ... 
9537    3.0
9538    NaN
9539    2.0
9540    3.0
9541    4.0
9542    NaN
9543    2.0
9544    NaN
9545    4.0
9546    3.0
9547    4.0
9548    2.0
9549    2.0
9550    4.0
9551    NaN
9552    4.0
9553    4.0
9554    3.0
9555    2.0
9556    3.0
9557    NaN
9558    2.0
9559    2.0
9560    NaN
9561    NaN
9562    2.0
9563    4.0
9564    4.0
9565    4.0
9566    4.0
Name: RegionsOOE, Length: 9567, dtype: float64

In [24]:
nnan = train_ooe.isnull()['RegionsOOE'].sum()
nnan

1999

In [30]:
temp = fill_nan('RegionsOOE', train_ooe)
temp = fill_nan('Region', temp)
train_ooe = temp
temp

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,Streaming-Applications,Terminals,Unclassified,VoIP,Web-Applications,CustomerAge,Region,Province,Product,RegionsOOE
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,45.0,16.0,14.0,0,2.0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,0.071551,0.000000,0.001896,0.000058,0.775410,25.0,0.0,NaN,0,2.0
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,0.099438,0.000000,0.012055,0.003579,0.499158,35.0,0.0,NaN,0,2.0
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,0.041474,0.000000,0.051692,0.013263,0.301528,55.0,8.0,15.0,2,2.0
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,0.005975,0.000000,0.000857,0.019149,0.926208,65.0,19.0,103.0,0,2.0
5,4483,1.0,0.155086,0.054729,1.0,0.001953,0.000499,0.036664,0.0,0.0,...,0.070697,0.000000,0.003048,0.001291,0.896900,55.0,0.0,NaN,0,2.0
6,7586,1.0,0.035019,0.010514,1.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,45.0,14.0,22.0,0,4.0
7,828,1.0,0.154708,0.010514,1.0,0.092448,0.000000,0.005903,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,7.0,37.0,0,2.0
8,12381,0.0,0.156087,0.024777,1.0,0.030599,0.000000,0.029139,0.0,0.0,...,0.105768,0.000000,0.001331,0.000036,0.760074,25.0,9.0,2.0,3,3.0
9,6377,0.0,NaN,0.010514,1.0,0.047526,0.004494,0.007696,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,75.0,8.0,56.0,0,2.0


In [98]:
train_ooe

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,Security,Streaming-Applications,Terminals,Unclassified,VoIP,Web-Applications,CustomerAge,Region,Province,Product
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.0,2.0,14.0,0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,0.002620,0.071551,0.000000,0.001896,0.000058,0.775410,25.0,2.0,NaN,0
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,0.000228,0.099438,0.000000,0.012055,0.003579,0.499158,35.0,2.0,NaN,0
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,0.000052,0.041474,0.000000,0.051692,0.013263,0.301528,55.0,2.0,15.0,2
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,0.000777,0.005975,0.000000,0.000857,0.019149,0.926208,65.0,2.0,103.0,0
5,4483,1.0,0.155086,0.054729,1.0,0.001953,0.000499,0.036664,0.0,0.0,...,0.000569,0.070697,0.000000,0.003048,0.001291,0.896900,55.0,2.0,NaN,0
6,7586,1.0,0.035019,0.010514,1.0,0.000000,0.000000,0.000000,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,45.0,4.0,22.0,0
7,828,1.0,0.154708,0.010514,1.0,0.092448,0.000000,0.005903,1.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,25.0,2.0,37.0,0
8,12381,0.0,0.156087,0.024777,1.0,0.030599,0.000000,0.029139,0.0,0.0,...,0.009175,0.105768,0.000000,0.001331,0.000036,0.760074,25.0,3.0,2.0,3
9,6377,0.0,NaN,0.010514,1.0,0.047526,0.004494,0.007696,0.0,0.0,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,75.0,2.0,56.0,0


In [81]:
temp['FieldToAdd'].values[0]

980.0

In [77]:
import math
test = train_ooe
le = len(test)
ct = temp['FieldToAdd'][0]
j = 0
for i in range(le):
    if(math.isnan(test['Region'][i])):
        test['Region'][i] = temp.index[j]
        #print("ct: " + str(ct) + " region: " + str(count.index[j]) + " J: " + str(j))
        ct -=1
        if(ct <= 0.0):
            j += 1
            ct = temp['FieldToAdd'][j]

KeyError: 0.0

In [221]:
age_ooe = train_ooe['CustomerAge']
age_ooe.values

young_customer = [5,15,25]
young = []
adult_customer = [35,45,55]
adult = []
old_customer = [65,75,85]
old = []

for i in age_ooe.values:
    if i in young_customer:
        young.append(1)
        adult.append(0)
        old.append(0)
    elif i in adult_customer:
        young.append(0)
        adult.append(1)
        old.append(0)    
    elif i in old_customer:
        young.append(0)
        adult.append(0)
        old.append(1)
    else:
        young.append(np.nan)
        adult.append(np.nan)
        old.append(np.nan)
        
yo = pd.DataFrame({'Young_Customer' : young})
ad = pd.DataFrame({'Adult_Customer' : adult})
ol = pd.DataFrame({'Old_Customer' : old})

train_ooe = train_ooe.join(yo)
train_ooe = train_ooe.join(ad)
train_ooe = train_ooe.join(ol)

In [222]:
train_ooe

,ID,DeviceFlag4G,DataArpu,DataAllowanceContinuous,DeviceFlagSmartphone,MonthlyVoiceTrafficCount,MonthlySMSTrafficCount,MonthlyDataTraffic,CustomerGender,CustomerExpatriate,...,CustomerAge,Region,Province,Product,North_Region,Middle_Region,South_Region,Young_Customer,Adult_Customer,Old_Customer
0,10930,1.0,0.156221,0.010514,1.0,0.018229,0.001623,0.011007,1.0,0.0,...,45.0,16.0,14.0,0,1.0,0.0,0.0,0.0,1.0,0.0
1,10170,0.0,0.155086,0.054729,1.0,0.051432,0.001498,0.039382,0.0,0.0,...,25.0,NaN,NaN,0,NaN,NaN,NaN,1.0,0.0,0.0
2,1492,1.0,NaN,NaN,1.0,0.058594,0.001248,0.033031,1.0,1.0,...,35.0,NaN,NaN,0,NaN,NaN,NaN,0.0,1.0,0.0
3,7424,1.0,0.155086,0.004987,1.0,0.097005,0.000499,0.002928,0.0,0.0,...,55.0,8.0,15.0,2,1.0,0.0,0.0,0.0,1.0,0.0
4,4332,1.0,0.155086,0.038148,1.0,0.034505,0.000375,0.001002,1.0,0.0,...,65.0,19.0,103.0,0,1.0,0.0,0.0,0.0,0.0,1.0
5,4483,1.0,0.155086,0.054729,1.0,0.001953,0.000499,0.036664,0.0,0.0,...,55.0,NaN,NaN,0,NaN,NaN,NaN,0.0,1.0,0.0
6,7586,1.0,0.035019,0.010514,1.0,0.000000,0.000000,0.000000,0.0,0.0,...,45.0,14.0,22.0,0,0.0,0.0,1.0,0.0,1.0,0.0
7,828,1.0,0.154708,0.010514,1.0,0.092448,0.000000,0.005903,1.0,0.0,...,25.0,7.0,37.0,0,1.0,0.0,0.0,1.0,0.0,0.0
8,12381,0.0,0.156087,0.024777,1.0,0.030599,0.000000,0.029139,0.0,0.0,...,25.0,9.0,2.0,3,0.0,1.0,0.0,1.0,0.0,0.0
9,6377,0.0,NaN,0.010514,1.0,0.047526,0.004494,0.007696,0.0,0.0,...,75.0,8.0,56.0,0,1.0,0.0,0.0,0.0,0.0,1.0


In [223]:
train_ooe.isnull().sum()

ID                                   0
DeviceFlag4G                         0
DataArpu                          2225
DataAllowanceContinuous           1180
DeviceFlagSmartphone                 0
MonthlyVoiceTrafficCount           339
MonthlySMSTrafficCount             339
MonthlyDataTraffic                 339
CustomerGender                       0
CustomerExpatriate                   0
ZipCode                           1726
ChurnScore                         869
AirportConnectionsDuration           0
AirportConnectionsCount              0
StationConnectionsDuration           0
StationConnectionsCount              0
ParkingConnectionsDuration           0
ParkingConnectionsCount              0
File-Transfer                        0
Games                                0
Instant-Messaging-Applications       0
Mail                                 0
Music-Streaming                      0
Network-Operation                    0
P2P-Applications                     0
Security                 